### Download data and install required libraries 

In [ ]:
!git clone https://github.com/stasaki/DEcode.git

In [ ]:
%cd DEcode

In [ ]:
!pip install --upgrade setuptools
!pip2 install shap==0.27.0

### Set location of input data

In [3]:
# Set input data

# Gene expression matrix
deg_data_file = "./data/toy/Transcriptome/expdata.txt"

# Location of RNA features
mRNA_data_loc = "./data/toy/RNA_features/"
mRNA_annotation_data = ["POSTAR","TargetScan"]

# Location of promoter features
promoter_data_loc = "./data/toy/Promoter_features/"
promoter_annotation_data = ["GTRD"]

# Genes used for traning, validation, and testing
train_genes = "./data/toy/Gene_splits/train.txt.gz"
validate_genes = "./data/toy/Gene_splits/validate.txt.gz"
test_genes = "./data/toy/Gene_splits/test.txt.gz"    

# Location of hyper-parameter
params_loc='./pretrained/Tissue_gene_params.json'

# Output directory
outloc='./train_out/full_model/'

### Define main function

In [3]:
import os
import sys
sys.path.append('./functions/')
import data
import model_utils
import pandas as pd
import numpy as np
import json

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

def main(params):
    from datetime import datetime
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import load_model
    import numpy as np
    import metrics
    import layer_utils
    import network
    
    print(params)
    just_return_model=False
    
    # model parameters and learning parameters
    max_epoch = 100
    batch_size = 128
    
    # batch initialization
    train_steps, train_batches = data.batch_iter(X_mRNA_train.values[:,1],
                                                 X_promoter_train.values[:,1],
                                                 Y_train.values[:,1:],
                                                 batch_size,
                                                 shuffle=True)
    valid_steps, valid_batches = data.batch_iter(X_mRNA_validate.values[:,1],
                                                 X_promoter_validate.values[:,1],
                                                 Y_validate.values[:,1:],
                                                 batch_size,
                                                 shuffle=True)
    test_steps, test_batches = data.batch_iter(X_mRNA_test.values[:,1],
                                               X_promoter_test.values[:,1],
                                               Y_test.values[:,1:],
                                               batch_size,
                                               shuffle=True)

    # Paramters for network structure
    params['n_feature_mRNA']=X_mRNA_train.values[:,1][0].shape[0]
    params['n_feature_promoter']=X_promoter_train.values[:,1][0].shape[0]
    params['n_out'] = Y_train.values[:,1:].shape[1]
    
    # Define network structure
    model = network.define_network(params)
    
    # If you don't need to traning model and just want to have model structure
    if just_return_model:
        return model
    
    # Set callback functions to early stop training and save the best model so far
    time_stamp=datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                    ModelCheckpoint(outloc+time_stamp+'_model.h5', monitor='val_loss', verbose=0,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='min', period=1)]
    
    # Optimizing model
    result = model.fit_generator(train_batches, train_steps, 
                                 epochs=max_epoch,
                                 validation_data=valid_batches,
                                 validation_steps=valid_steps,
                                 callbacks=callbacks,
                                 max_queue_size=10,
                                 verbose=0)
    
    # Test performance
    # Load best model
    model = load_model(outloc+time_stamp+'_model.h5',
                   custom_objects={'pcor': metrics.pcor,
                                  'GlobalSumPooling1D': layer_utils.GlobalSumPooling1D})
    test_performance= np.array(model.evaluate_generator(test_batches,test_steps))
    np.savetxt(outloc+time_stamp+'_test_performance.txt',
               test_performance,delimiter="\t")
    
    # Saving optimization history
    with open(outloc+time_stamp+'_history.json', 'w') as f:
        json.dump(result.history, f)
    
    # Saving model and learning paramters
    with open(outloc+time_stamp+'_params.json', 'w') as f:
        json.dump(params, f)
    
    # Return validation loss for model selection
    validation_loss = np.amin(result.history['val_loss']) 
    
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

### Training model

In [ ]:
! mkdir -p "$outloc"
shuffle="None"
# Prepare learning data
Y_train, Y_validate, Y_test, X_mRNA_train, X_mRNA_validate, X_mRNA_test, X_promoter_train, X_promoter_validate, X_promoter_test = data.prep_ml_data_split(
    deg_data_file=deg_data_file,
    mRNA_data_loc=mRNA_data_loc,
    mRNA_annotation_data=mRNA_annotation_data,
    promoter_data_loc=promoter_data_loc,
    promoter_annotation_data=promoter_annotation_data,
    train_genes=train_genes,
    validate_genes=validate_genes,
    test_genes=test_genes,
    outloc=outloc,
    shuffle=shuffle)

# Obtain hyper parameters
with open(params_loc) as f:
    params=json.load(f)  

# Training model
from hyperopt import STATUS_OK
for i in range(10):
    main(params)

# Summarizing the result
! Rscript functions/find_best_model.R "$outloc" &> /dev/null 
with open(outloc+"/summary/best_model.txt") as f:
    best_model=f.readline().rstrip()

### Compute Spearman's correlation between actual and predicted expression for each sample

In [ ]:
# Prediction for test samples with the best model
model_utils.test_prediction(outloc,
                            best_model,
                            X_mRNA_test,
                            X_promoter_test,
                            Y_test)

In [19]:
! Rscript --vanilla --slave functions/calc_performance.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/test_data/cor_tbl.txt',sep="\t")

,sample,estimate,statistic,p.value,method,alternative
0,Adipose_Subcutaneous,0.036083,3.162086e+06,5.549578e-01,Spearman's rank correlation rho,two.sided
1,Adipose_Visceral_Omentum,0.059983,3.083684e+06,3.261347e-01,Spearman's rank correlation rho,two.sided
2,Adrenal_Gland,0.202853,2.615006e+06,8.003660e-04,Spearman's rank correlation rho,two.sided
3,Artery_Aorta,0.093184,2.974770e+06,1.266649e-01,Spearman's rank correlation rho,two.sided
4,Artery_Coronary,0.219531,2.560292e+06,2.780077e-04,Spearman's rank correlation rho,two.sided
5,Artery_Tibial,0.309306,2.265792e+06,2.142011e-07,Spearman's rank correlation rho,two.sided
6,Bladder,0.104497,2.937656e+06,8.656588e-02,Spearman's rank correlation rho,two.sided
7,Brain_Amygdala,0.391838,1.995047e+06,2.428589e-11,Spearman's rank correlation rho,two.sided
8,Brain_Anterior_cingulate_cortex_BA24,0.385926,2.014441e+06,5.092438e-11,Spearman's rank correlation rho,two.sided
9,Brain_Caudate_basal_ganglia,0.320192,2.230080e+06,7.483287e-08,Spearman's rank correlation rho,two.sided


### Compute average DeepLIFT score for each regulator

In [ ]:
# Estimate variable imporance using test samples
model_utils.compute_DeepLIFT(outloc,
                             best_model,
                             X_mRNA_test,
                             X_promoter_test,
                             Y_test)

In [24]:
! Rscript --vanilla --slave functions/summarize_DeepLIFT.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/DeepLIFT/RNA_importance_mean.txt',sep="\t")
pd.read_csv(outloc+best_model+'/DeepLIFT/promoter_importance_mean.txt',sep="\t")

,sample_name,AEBP2,AHR,AHRR,APC,AR,ARID1A,ARID1B,ARID2,ARID3A,...,ZSCAN21,ZSCAN22,ZSCAN29,ZSCAN31,ZSCAN4,ZSCAN5A,ZSCAN5DP,ZSCAN9,ZXDB,ZXDC
0,Adipose_Subcutaneous,-0.000574,-0.008066,-0.001932,0.002497,-0.035218,-0.001208,0.005083,-0.001784,-0.000488,...,0.000247,-0.001703,-0.001132,0.000012,0.001092,0.002686,0.001755,0.000890,0.001437,-0.001689
1,Adipose_Visceral_Omentum,0.000317,-0.003069,-0.000966,-0.000329,-0.075674,0.000066,0.002007,-0.000364,-0.000356,...,0.000342,-0.003087,-0.001388,-0.000034,0.000782,0.001071,0.001396,-0.003565,-0.000279,0.001391
2,Adrenal_Gland,0.000841,0.005638,0.000666,0.000879,-0.041112,-0.001166,-0.002949,0.000961,-0.000543,...,0.000199,-0.012073,0.000828,0.000473,0.000856,-0.001733,-0.000201,0.000823,-0.000497,0.005168
3,Artery_Aorta,-0.001321,-0.007359,-0.001278,0.001188,0.006742,-0.000874,-0.000215,-0.001236,0.001894,...,0.000109,0.004866,-0.000202,-0.000657,0.001273,0.001864,-0.000358,-0.003479,-0.000027,-0.003910
4,Artery_Coronary,-0.000551,-0.003065,-0.000684,0.000660,0.016316,0.000328,0.002592,-0.001322,-0.000412,...,0.000195,0.002899,-0.001751,-0.000527,0.000471,0.003176,-0.000002,-0.001534,0.002213,0.000653
5,Artery_Tibial,-0.002034,-0.008305,-0.001103,0.003824,0.054916,-0.000385,0.002311,-0.001709,0.002887,...,0.000190,0.007613,0.001010,-0.000683,0.002154,0.003218,-0.001018,-0.006260,0.001995,-0.000652
6,Bladder,0.000869,-0.004921,-0.001135,0.002005,-0.006714,-0.001053,0.005836,-0.000216,-0.001640,...,0.000433,-0.004932,-0.000286,0.000580,-0.001168,0.000191,0.001625,0.001273,-0.000582,0.004917
7,Brain_Amygdala,-0.001129,0.012314,0.004060,-0.003634,0.088127,0.001923,-0.014325,0.000172,0.001905,...,-0.000976,-0.015159,0.002596,-0.000527,-0.002280,-0.005034,-0.005361,0.001297,0.003560,-0.004163
8,Brain_Anterior_cingulate_cortex_BA24,-0.000694,0.011971,0.004595,-0.003949,0.093863,0.002962,-0.015312,0.001084,0.002259,...,-0.000944,-0.020171,0.002934,-0.000699,-0.004116,-0.006219,-0.006089,0.004160,0.001481,-0.001249
9,Brain_Caudate_basal_ganglia,-0.000702,0.016507,0.004531,-0.006676,0.090035,0.001621,-0.016353,0.001200,0.002594,...,-0.001006,-0.011742,0.001949,-0.000608,-0.003142,-0.005498,-0.004746,0.001272,0.002299,-0.002939


### Simulate the concequence of regulator knockout

In [ ]:
# Simulate the concequence of regulator knockout
genes=["ENSG00000268903","ENSG00000239906"]
model_utils.coexpression_with_KO(genes,
                                 outloc,
                                 best_model,
                                 X_mRNA_test,
                                 X_promoter_test,
                                 Y_test)    

In [27]:
! Rscript --vanilla --slave functions/test_KO.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/regulator_KO/regression_res.txt',sep="\t")

,term,estimate,std.error,statistic,p.value
0,ELAVL1,-0.036899,0.001287,-28.675414,9.588536e-174
1,CTCF,-0.036143,0.001289,-28.045841,1.440294e-166
2,CEBPB,0.033227,0.001287,25.809119,3.545716e-142
3,ZSCAN5A,-0.029915,0.001289,-23.199254,5.815857e-116
4,PRPF8,-0.029156,0.001287,-22.654990,8.289859e-111
5,BUD13,-0.028493,0.001291,-22.062869,2.516819e-105
6,KMT2A,-0.028332,0.001285,-22.042022,3.903797e-105
7,ZNF549,0.026184,0.001291,20.282094,1.243803e-89
8,CDX2,0.025479,0.001287,19.798494,1.407658e-85
9,DDX3X,-0.024187,0.001289,-18.763255,3.337379e-77


### Simulate the concequence of binding site removals

In [ ]:
# Simulate the concequence of binding site removals
genes=["ENSG00000268903","ENSG00000239906"]
model_utils.coexpression_with_binding_site_removal(genes,
                                                   outloc,
                                                   best_model,
                                                   X_mRNA_test,
                                                   X_promoter_test,
                                                   Y_test)

In [28]:
! Rscript --vanilla --slave functions/test_interval.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/binding_site_removal/regression_res.txt',sep="\t")

,term,estimate,std.error,statistic,p.value,Gene
0,RNA_interval_3,-4.956493e-02,0.000873,-56.803320,0.000000e+00,ENSG00000196660.6
1,promoter_interval_7,-3.736345e-02,0.000873,-42.788975,0.000000e+00,ENSG00000196660.6
2,promoter_interval_13,3.659978e-02,0.000873,41.941760,0.000000e+00,ENSG00000196660.6
3,RNA_interval_1,-5.517174e-02,0.000209,-264.192164,0.000000e+00,ENSG00000243000.1
4,promoter_interval_1,-7.020380e-02,0.000209,-336.476465,0.000000e+00,ENSG00000243000.1
5,promoter_interval_20,1.124924e-02,0.000209,53.868172,0.000000e+00,ENSG00000243000.1
6,promoter_interval_26,2.776770e-02,0.000209,132.967189,0.000000e+00,ENSG00000243000.1
7,promoter_interval_27,8.724134e-02,0.000209,417.670434,0.000000e+00,ENSG00000243000.1
8,promoter_interval_28,1.343480e-02,0.000209,64.425864,0.000000e+00,ENSG00000243000.1
9,promoter_interval_15,-3.075650e-02,0.000873,-35.225559,3.260689e-256,ENSG00000196660.6
